In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

from data_processing import split
from utils import diff
import random

In [2]:
from sklearn.metrics import roc_auc_score

In [3]:
df = pd.read_csv('./data/trainnf.csv')

In [4]:
df.head()

,Id,physics,chemistry,biology,log_mean_exam_points,sqr_mean_exam_points,mean_exam_points,sqrt_mean_exam_points,sqrt_lesson_price,lesson_price,sqr_lesson_price,log_lesson_price,choose
0,0.0,0.0,0.0,0.0,0.728407,0.492313,0.611940,0.671556,0.661630,0.520000,0.294458,0.796106,0.0
1,1.0,1.0,0.0,1.0,0.492976,0.242397,0.358209,0.424227,0.435528,0.280000,0.097831,0.614310,1.0
2,2.0,1.0,0.0,0.0,0.625210,0.366626,0.492537,0.559161,0.568521,0.413333,0.194217,0.727101,0.0
3,3.0,0.0,0.0,0.0,0.625210,0.366626,0.492537,0.559161,0.374928,0.226667,0.068273,0.555864,1.0
4,4.0,1.0,0.0,0.0,0.716135,0.475816,0.597015,0.657850,0.683518,0.546667,0.322731,0.811345,0.0


In [5]:
TARGET_NAME = 'choose'
X_train, X_test, y_train, y_test = split(df, diff(list(df.columns), [TARGET_NAME]), TARGET_NAME)

In [6]:
random.seed(42)

def get_bootstrap(data, labels, N):
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        
        for j in range(n_samples):
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]
            
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [7]:
def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = [i for i in range(len_sample)]
    
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    
    random.shuffle(sample_indexes)
    for _ in range(len_subsample):
        subsample.append(sample_indexes.pop())
        
    return subsample

In [8]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле
        
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction
    
def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

def quality(left_labels, right_labels, current_gini):
    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 1

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels))
        
    return forest

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)
    
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))
        
    return voted_predictions

In [9]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [10]:
forest = random_forest(X_train.values[:500], y_train.values[:500], 7)

In [11]:
train_answers = tree_vote(forest, X_train.values)
test_answers = tree_vote(forest, X_test.values)

print(roc_auc_score(y_train.values, train_answers))
print(roc_auc_score(y_test.values, test_answers))

0.5959655155097985
0.5952046647373327


In [12]:
df_test = pd.read_csv('./data/testnf.csv')

In [13]:
df_test.head()

,Id,physics,chemistry,biology,log_mean_exam_points,sqr_mean_exam_points,mean_exam_points,sqrt_mean_exam_points,sqrt_lesson_price,lesson_price,sqr_lesson_price,log_lesson_price
0,10000.0,0.0,0.0,0.0,0.923914,0.824709,0.878788,0.902735,0.760864,0.657534,0.464142,0.852401
1,10001.0,1.0,1.0,0.0,0.712046,0.468182,0.590909,0.652729,0.551432,0.410959,0.203062,0.695103
2,10002.0,0.0,0.0,0.0,0.304608,0.116667,0.196970,0.247805,0.380432,0.246575,0.087027,0.537805
3,10003.0,1.0,0.0,0.0,0.943552,0.867133,0.909091,0.927443,0.812532,0.726027,0.555197,0.886755
4,10004.0,1.0,0.0,0.0,0.531353,0.272727,0.393939,0.461722,0.441395,0.301370,0.120548,0.597607


In [14]:
test_answers_ = tree_vote(forest, df_test.values)

In [15]:
df_test['Id'] = df_test['Id'].astype(int)
df_test.head()

,Id,physics,chemistry,biology,log_mean_exam_points,sqr_mean_exam_points,mean_exam_points,sqrt_mean_exam_points,sqrt_lesson_price,lesson_price,sqr_lesson_price,log_lesson_price
0,10000,0.0,0.0,0.0,0.923914,0.824709,0.878788,0.902735,0.760864,0.657534,0.464142,0.852401
1,10001,1.0,1.0,0.0,0.712046,0.468182,0.590909,0.652729,0.551432,0.410959,0.203062,0.695103
2,10002,0.0,0.0,0.0,0.304608,0.116667,0.196970,0.247805,0.380432,0.246575,0.087027,0.537805
3,10003,1.0,0.0,0.0,0.943552,0.867133,0.909091,0.927443,0.812532,0.726027,0.555197,0.886755
4,10004,1.0,0.0,0.0,0.531353,0.272727,0.393939,0.461722,0.441395,0.301370,0.120548,0.597607


In [16]:
df_test_answers = pd.DataFrame({'Id':df_test['Id'].values, 'choose':test_answers_})
df_test_answers.head()

,Id,choose
0,10000,0.0
1,10001,0.0
2,10002,0.0
3,10003,0.0
4,10004,0.0


In [17]:
df_test_answers.to_csv('./data/test_answers.csv', index=False)